In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization, Dropout, LSTM, TimeDistributed
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Reshape

from sklearn.model_selection import train_test_split
import numpy as np
import json

# Load data
with open('../Data/data.json', 'r') as fp:
    data = json.load(fp)

# Extract MFCCs and labels
mfccs = data['mfcc']
labels = data['genre_num']

# Validate and align data
print(f"Original MFCCs length: {len(mfccs)}, Labels length: {len(labels)}")
min_length = min(len(mfccs), len(labels))
mfccs = mfccs[:min_length]
labels = labels[:min_length]
print(f"Aligned MFCCs length: {len(mfccs)}, Labels length: {len(labels)}")

# Convert to NumPy arrays
X = np.array(mfccs)
y = np.array(labels)

# Train-validation-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=42, stratify=y_train)

# Reshape for CRNN
X_train = X_train[..., np.newaxis]
X_val = X_val[..., np.newaxis]
X_test = X_test[..., np.newaxis]
input_shape = X_train.shape[1:4]

# Define CRNN model
model_crnn = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=input_shape),
    BatchNormalization(),
    MaxPooling2D((2, 2), padding='same'),
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2), padding='same'),
    # Reshape to (time_steps, features) for LSTM
    Reshape((-1, 128)),  # -1 infers the time dimension, 128 is the number of filters in the last Conv2D
    LSTM(64, return_sequences=False),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])
# Compile model
model_crnn.compile(
    optimizer=Adam(learning_rate=0.00051),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

# Data augmentation
datagen = ImageDataGenerator(vertical_flip=True)

# Train model
model_crnn.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    validation_data=(X_val, y_val),
    epochs=100,
    callbacks=[early_stopping],
    verbose=1
)

# Evaluate
loss, accuracy = model_crnn.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")


Original MFCCs length: 9986, Labels length: 9986
Aligned MFCCs length: 9986, Labels length: 9986
Epoch 1/100
153/153 [==============================] - 15s 82ms/step - loss: 1.8419 - accuracy: 0.3417 - val_loss: 1.6265 - val_accuracy: 0.3915
Epoch 2/100
153/153 [==============================] - 12s 77ms/step - loss: 1.4515 - accuracy: 0.4715 - val_loss: 1.3678 - val_accuracy: 0.5007
Epoch 3/100
153/153 [==============================] - 13s 84ms/step - loss: 1.2893 - accuracy: 0.5342 - val_loss: 1.1677 - val_accuracy: 0.5994
Epoch 4/100
153/153 [==============================] - 12s 76ms/step - loss: 1.1938 - accuracy: 0.5792 - val_loss: 1.1491 - val_accuracy: 0.6018
Epoch 5/100
153/153 [==============================] - 12s 76ms/step - loss: 1.0823 - accuracy: 0.6190 - val_loss: 1.0236 - val_accuracy: 0.6352
Epoch 6/100
153/153 [==============================] - 11s 74ms/step - loss: 1.0519 - accuracy: 0.6284 - val_loss: 0.9890 - val_accuracy: 0.6643
Epoch 7/100
153/153 [============

In [5]:
# Save model
model_crnn.save('../models/crnn.keras')